<a href='https://ai.meng.duke.edu'> = <img align="left" style="padding-top:10px;" src=https://storage.googleapis.com/aipi_datasets/Duke-AIPI-Logo.png>

# Visualization of Inspiration Astronaut Biomarkers After 3 Days in Space

## Background
We are acting as the first data scientist on the Inspiration-Health project. Our first major task is to build a reproducible pipeline to clean, transform, and visualize astronaut biomarker data from the SpaceX Inspiration4 mission.
Instead of building predictive models, our focus is on identifying and visualizing significant deviations from baseline values to help communicate health changes during and after spaceflight.


## Data
We are working with astronaut biospecimen data from the SpaceX Inspiration4 Mission (NASA OSD-575).
The cleaned files in `cleaned_data/` include the Comprehensive Metabolic Panel (CMP), Cardiovascular Multiplex Panel, and Immune Multiplex Panel.

Each dataset is organized in long format, where each row corresponds to a single measurement for a given astronaut at a given timepoint.
Key columns include:

- **analyte** – the biomarker name (e.g., *GLUCOSE, CREATININE, SODIUM, ALBUMIN*).
- **value** – measured biomarker value.
- **range_min**, **range_max** – clinical reference ranges for each analyte. Depending on the analyte this is the minimum or maximum detectable or it referred to healthy reference ranges.
- **units** – measurement units (mg/dL, U/L, etc.).
- **test_type** – indicates panel (CMP, cardiovascular multiplex, immune multiplex).
- **subject_id** – astronaut ID (C001–C004).
- **sex** – M/F.
- **timepoint** – collection time relative to flight (*L-92, L-44, L-3 pre-flight*; *R+1, R+45, R+82 post-flight*).

These features allow us to:
- Track biomarker trajectories across mission phases.
- Compare astronaut values to **baseline (pre-flight average)**.
- Identify deviations outside clinical reference ranges.
- Visualize individual astronaut responses and group trends.

## Approach

We will build a pipeline that:

1. **Load Data**
   Merge metadata and biomarker tables.

2. **Clean Data**
   Handle missing values, outliers, and consistency issues.

3. **Identify Data by Phase**
   Group values into pre-flight vs. post-flight to allow comparison.

4. **Feature Engineering for Visualization**
   - Compute **baseline mean** for each analyte (average of L-92, L-44, L-3).
   - Calculate **delta = value – baseline** at each post-flight timepoint.
   - Flag analytes with statistically or clinically notable deviations.

5. **Visualization**
   - Line plots of biomarker trajectories across timepoints.
   - Highlight analytes with the largest relative changes.

6. **Communication**
   Produce visuals and narratives that explain *what changed, when it changed, and why it matters* in accessible language.


In [ ]:
# Run this before any other code cell
# This downloads the csv data files into the same directory where you have saved this notebook
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# ==== CONFIG ====
DATA_DIR = Path("cleaned_data")

# Files (pick the one you want to analyze)
CMP_FILE      = DATA_DIR / "Metabolic_Panel.csv"
CARDIO_FILE   = DATA_DIR / "Serum_Cardiovascular.csv"
IMMUNE_FILE   = DATA_DIR / "LSDS-8_Multiplex_serum_immune_EvePanel_TRANSFORMED_all_astronauts.csv"

# ==== LOAD DATA ====
def load_data(file):
    df = pd.read_csv(file)
    return df

cmp = load_data(CMP_FILE)
cmp.head()

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LinearRegression
from sklearn import metrics
import pandas as pd

def run_model(X_train,y_train,X_test,y_test):

    lin_model = LinearRegression()
    lin_model.fit(X_train,y_train)
    y_pred = lin_model.predict(X_test)
    test_mse = metrics.mean_squared_error(y_test, y_pred)
    return test_mse

import warnings
warnings.filterwarnings("ignore")

pd.options.display.float_format = '{:,.2f}'.format

### Load data
Create and run a function `load_data()` to do your data loading and any merging needed.  You can specify the arguments and returns as needed.

In [ ]:
def load_data(file1, file2):
    df1 = pd.read_csv(file1)
    df2 = pd.read_csv(file2)
    merged = pd.merge(df1, df2, how="outer", on=['dteday','hr'])
    return merged

### Preprocessing the data

The data files were preprocessed using `preprocess.py`

This script automates preprocessing of astronaut health data stored in CSV files. It reads raw transformed data from the `data` directory, extracts astronaut identifiers and timepoints, checks for missing/duplicate values, and outputs cleaned datasets in both combined and astronaut-specific formats.

#### Features
1. **Reads all CSV files** in the specified input directory (`data` by default).  
2. **Splits `Sample Name`** column (e.g., `C001_serum_L-3`) into:  
   - `astronautID` (e.g., `C001`)  
   - `timepoint` (e.g., `L-3`)  
3. **Quality checks:**  
   - Reports missing values per column.  
   - Reports duplicate rows.  
4. **Saves outputs:**  
   - `<filename>_all_astronauts.csv` → combined file with astronaut/timepoint columns added.  
   - `<filename>_<astronautID>.csv` → per-astronaut subsets for easier analysis.

---

#### Expected Input Format
- CSV files must contain a column named **`Sample Name`**.  
- Expected naming convention in `Sample Name`:  

### Dealing with Missing Values

During preprocessing of the metabolic panel data some columns contained missing values.  
The following imputation strategy is applied in analysis:

| Column                                                                 | Missing Count | Strategy          | Rationale                                   |
|------------------------------------------------------------------------|----------------|-------------------|---------------------------------------------|
| `total_bilirubin_value_milligram_per_deciliter`                        | 1              | Median imputation | Bilirubin is skewed, median is more robust. |
| `bun_to_creatinine_ratio_value`                                        | 22             | Median imputation | Ratio, skewed distribution.                 |
| `bun_to_creatinine_ratio_range_min`                                    | 1              | Median imputation | Safe choice for small gaps.                 |
| `bun_to_creatinine_ratio_range_max`                                    | 1              | Median imputation | Safe choice for small gaps.                 |
| `egfr_african_american_range_max_milliliter_per_minute_per_1.73m²`     | 28             | Dropped  | All values missing         |
| `egfr_non_african_american_range_max_milliliter_per_minute_per_1.73m²` | 28             | Dropped   | All values missing         |

#### Why this approach?
- **Median** is robust to skewed lab values and avoids distortion by outliers.  

### Outlier Detection in Metabolic Panel

We applied **boxplot-based outlier detection** (IQR method) across all astronauts combined.  

####  Method
- For each numeric biomarker:
  - Compute **Q1 (25th percentile)** and **Q3 (75th percentile)**.  
  - Calculate **IQR = Q3 − Q1**.  
  - Define bounds:
    - **Lower Bound = Q1 − 1.5 × IQR**  
    - **Upper Bound = Q3 + 1.5 × IQR**  
  - Any value outside these bounds is flagged as an **outlier**.

#### Visualization
- Boxplots are generated for each biomarker.  
- Outlier points are **annotated with astronautID and timepoint** for traceability.    
- A boxplot is only shown if the column contains at least one outlier (to reduce clutter).

### Feature Engineering

Once we had our `final_data`, we applied the `featureEngineering.py` script to enrich the datasets with two key transformations:

1. Numeric flight-day scale
   - Raw `timepoint` labels like `L-3` and and `R+1` are mapped onto a stretched numeric axis.
   - Launch day (`L0`) is set to day 0.
   - Pre-launch days count down into the negatives (e.g., `L-3 → -3`).
   - In-flight is artificially stretched to **30 days** (`R+0 → 30`), so short spaceflights appear visibly distinct from recovery.
   - Recovery days then continue sequentially (`R+1 → 31`, `R+7 → 37`, etc.).
   - This stretching is intentional: while it introduces “fake” days, it ensures plots visually distinguish pre-launch**, in-flight, and post-flight phases.

2. Derived features
   - We compute the Anion Gap as:
     $$
     \text{Anion Gap} = [\text{Na}^+] - [\text{Cl}^-] - [\text{CO}_2]
     $$
   - This metric is clinically relevant for detecting metabolic acidosis and other imbalances, and is not directly provided in the raw data.
   - The function adds `anion_gap_value` along with placeholder `min`/`max` columns so the reference ranges can be defined in `stats.py`.

Together, these functions clean redundant columns (like `Sample Name`), provide a consistent timeline across astronauts, and introduce clinically meaningful derived biomarkers. This ensures downstream analysis and visualization are both aligned and interpretably enriched.


In [ ]:
import pandas as pd
from scripts.featureEngineering import add_flight_day, add_derived_features
from scripts.stats import tidy_from_wide

# Load cleaned metabolic panel
df = pd.read_csv("final_data/Metabolic_Panel.csv")

# Feature engineering
df = add_flight_day(df)
df = add_derived_features(df)

# Convert to tidy format
tidy_df = tidy_from_wide(df)

### Identify All the Statistical Deviance

We want a simple table of analytes where an individual astronaut’s R+1 value was statistically higher than baseline (p < 0.05 and R+1 > mean baseline).

The code below takes the results from `analyze_r1_vs_L()` and filters for these conditions, then saves the table to CSV for easy inspection.

---

#### Statistical Background

For each astronaut $(a)$ and analyte $(x)$:

1. Baseline distribution
   Collect all pre-flight measurements (timepoints \(L-92, L-44, L-3\)):

   $$
   L_{a,x} = \{ v_{a,x}^{L-92}, v_{a,x}^{L-44}, v_{a,x}^{L-3} \}
   $$

   with sample mean

   $$
   \bar{L}_{a,x} = \frac{1}{n}\sum_{i=1}^{n} v_{a,x}^{L_i}
   $$

   and sample standard deviation

   $$
   s_{a,x} = \sqrt{\frac{1}{n-1}\sum_{i=1}^{n}(v_{a,x}^{L_i}-\bar{L}_{a,x})^2}.
   $$

2. R+1 observation
   Take the recovery-day measurement:

   $$
   R1_{a,x} = v_{a,x}^{R+1}.
   $$

3. One-sample t-test
   Null hypothesis $(H_0)$: astronaut’s R+1 value is consistent with baseline distribution.

   Test statistic:

   $$
   t = \frac{R1_{a,x} - \bar{L}_{a,x}}{s_{a,x}/\sqrt{n}}
   $$

   where $n = |L_{a,x}|$.
   The p-value is computed from Student’s $t$-distribution with $n-1$ degrees of freedom.

4. Effect size (Cohen’s d)
   To measure magnitude:

   $$
   d = \frac{R1_{a,x} - \bar{L}_{a,x}}{s_{a,x}}
   $$

---

#### Why This Matters

- A significant p-value $(<0.05)$ means it’s unlikely the observed R+1 value arose from random baseline variation.
- A positive Cohen’s d indicates the R+1 measurement is elevated above baseline, with larger values meaning stronger effects.
- By filtering for cases where $(R1_{a,x} > \bar{L}_{a,x})$, we isolate biomarkers that spike upward at R+1, which may signal stress, inflammation, or recovery physiology.



In [ ]:
from scripts.stats import analyze_r1_vs_L, ANALYTE_INFO

# Now analyze
results_df = analyze_r1_vs_L(tidy_df)

elevated = results_df[
    (results_df["test_type"] == "within") &
    (results_df["p_value"] < 0.05) &
    (results_df["R1"] > results_df["mean_L"])
].copy()

elevated["label"] = elevated["analyte"].map(lambda x: ANALYTE_INFO[x]["label"])
elevated["unit"] = elevated["analyte"].map(lambda x: ANALYTE_INFO[x]["unit"])

# Reorder for readability
elevated = elevated[
    ["astronautID", "analyte", "label", "unit", "mean_L", "R1", "p_value", "effect_size"]
]

# Save to CSV
elevated.to_csv("R1_elevated_analytes.csv", index=False)

elevated.head()

### Feature Engineering
Create and run the function `build_features()` below to create any additional derivative features (e.g. time series features) that you wish to use in modeling.  You will need to apply this function to both your training and test sets.
#### Stats
This module provides statistical processing functions
for astronaut serum and biochemical datasets.

##### Overview
--------
1. Defines a consistent analyte metadata map (`ANALYTE_INFO`)
   that assigns each analyte a human-friendly label and unit.

2. Transforms the wide-format astronaut dataset
   (value, min, max triplets for each analyte)
   into a tidy long-format DataFrame for analysis and plotting.

3. Provides statistical comparison functions to test
   whether recovery day 1 (R+1) is significantly shifted
   relative to the pre-launch baseline (L-series).

##### Data Model
----------
The tidy DataFrame returned by `tidy_from_wide` has the following columns:

    astronautID   : string, subject identifier (e.g. "C001")
    timepoint     : string, raw timepoint (e.g. "L-3", "R+1")
    flight_day    : integer, absolute scale where:
                      L-0 = 0 (launch day)
                      R+0 = 3 (last day in space)
                      R+1 = 4 (first recovery day)
                      L-n = -n (pre-launch)
    analyte       : string, machine-readable analyte name
    value         : numeric, observed measurement
    min           : numeric, reference range minimum
    max           : numeric, reference range maximum
    label         : string, human-readable analyte label
    unit          : string, measurement unit

##### Statistical Tests
-----------------
`analyze_r1_vs_L(tidy_df)` applies two complementary analyses:

1. Within-astronaut analysis
   - For each astronaut and analyte, compares the R+1 measurement
     to that astronaut’s baseline distribution of L-values.
   - Implemented as a one-sample t-test:
         H0: R+1 = mean(L)
   - Reports mean baseline, R+1, t-statistic, p-value,
     and effect size (Cohen’s d).

2. Across-astronaut analysis
   - Aggregates astronauts by computing each subject’s baseline mean (L-mean).
   - Performs a paired t-test across astronauts:
         H0: mean(L-means) = mean(R+1)
   - Reports group means, t-statistic, p-value,
     and effect size (mean difference / SD of differences).

##### Returned Output
---------------
The results are returned as a DataFrame with columns:

    analyte      : analyte name
    astronautID  : subject ID ("ALL" for group-level test)
    test_type    : "within" (per astronaut) or "group" (across astronauts)
    n_L          : number of L timepoints used
    mean_L       : baseline mean
    R1           : R+1 value (or mean across astronauts)
    t_stat       : test statistic
    p_value      : two-sided p-value
    effect_size  : Cohen’s d

##### Use Cases
---------
- Identify analytes that change significantly at R+1 relative to baseline.
- Assess subject-specific vs. group-level recovery patterns.
- Provide both inferential results (p-values) and effect sizes
  for scientific reporting and visualization

In [ ]:
# Stats

### Feature Selection
Use the cell below to create and run the function `feature_select()` which performs feature selection using univariate (filter) methods.  After you analyze the correlations, determine whether you would like to remove any features and do so.

### Prepare Features for Modeling
Our final step in the pipeline is to prepare our feature set for modeling.  In particular, in this step we need to ensure that any categorical variables we may be using are encoded as numeric values in order for the model to function properly.  You might also consider scaling some of your data.

In the below cell create and run a function `prepare_train_feats()` which prepares the training features.

We also need to prepare the features in our test set in the same way to feed into the model.  Use the cell below for the function `prepare_test_feats()` which prepares your test set features.

### Run pipeline
Finally, let's bring everything together in a function to run the entire pipeline for our training data.  Complete the function `run_pipeline()` in the cell below.  The function should call any/all of the functions you have defined above which are needed to load the data, transform it and prepare the features for both the training set and the test set.

In [ ]:
def run_pipeline(bike_filename, weather_filename):
    '''
    Runs your pipeline (calling the above functions as needed) to transform the raw data into the training and test data sets for modeling

    Inputs:
        bike_filename(str): name of the file containing the bike data
        weather_filename(str): name of the file containing the weather data

    Returns:
        X_train(pd.DataFrame): dataframe containing the training set inputs
        y_train(pd.DataFrame): dataframe containing the training set labels
        X_test(pd.DataFrame): dataframe containing the test set inputs
        y_test(pd.DataFrame): dataframe containing the test set labels
    '''
    merged = load_data(bike_filename,weather_filename)
    clean = clean_data(merged)
    X_train, y_train, X_test, y_test = split_data(clean)

    return X_train, y_train, X_test, y_test

    

Now that we've prepared our features we are ready to run our model.  Run the cell below, which trains the model on the training set and calculates and reports the mean squared error (MSE) on the test set.  If everything went well you should have a MSE below 18500

In [ ]:
bike_datafile = "2011-2012_bikes.csv"
weather_datafile = "2011-2012_weather_messy.csv"
X_train, y_train, X_test, y_test = run_pipeline(bike_datafile, weather_datafile)
mse_score = run_model(X_train, y_train, X_test, y_test)
print('Mean Squared Error on the test set: {:.2f}'.format(mse_score))

assert mse_score < 18500